In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import matplotlib as plt

In [15]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [16]:
df = pd.read_csv('fetal_health.csv')

In [17]:
def data_preprocessing(dataframe, target_col):
    df = dataframe.drop_duplicates()
    features = dataframe.drop(target_col, axis=1)
    label = dataframe[target_col]
    label = pd.get_dummies(data=label, columns=[target_col])
    columns = features.columns
    for col in columns:
        if type(features[col]) == str:
            features = pd.get_dummies(data=features, columns=[col])
        else:
            col_mean = np.mean(features[col])
            features[col] = features[col].fillna(col_mean)
    scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
    scaled_features = scaler.fit_transform(features)
    return scaled_features, label

In [111]:
X, Y = data_preprocessing(df, 'fetal_health')

In [322]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=7)
X_train, X_Dev, Y_train, Y_Dev = train_test_split(X_train, Y_train, test_size=0.2, random_state=7)
print(X_Dev.shape)

(298, 21)


In [363]:
initializer = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)

In [364]:
const_weight = tf.keras.constraints.MinMaxNorm(min_value=0.0, max_value=1.0, rate=1.0, axis=0)

In [365]:
model = keras.Sequential()
model.add(Dense(units=128, input_dim=X_train.shape[1], activation='relu', activity_regularizer='l1', bias_constraint=const_weight))
model.add(Dropout(0.5))
model.add(Dense(units=512, activation='relu', activity_regularizer='l2', bias_initializer=initializer, bias_constraint=const_weight))
model.add(Dropout(0.5))
model.add(Dense(units=256, activation='relu', activity_regularizer='l2', bias_initializer=initializer, bias_constraint=const_weight))
model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu', activity_regularizer='l2', bias_initializer=initializer, bias_constraint=const_weight))
model.add(Dense(units=3, activation='softmax', bias_constraint=const_weight))

In [366]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.5)
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', \
              optimizer=adam, metrics=['accuracy'])

In [367]:
print(model.summary())

Model: "sequential_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_225 (Dense)            (None, 128)               2816      
_________________________________________________________________
dropout_113 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_226 (Dense)            (None, 512)               66048     
_________________________________________________________________
dropout_114 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_227 (Dense)            (None, 256)               131328    
_________________________________________________________________
dropout_115 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_228 (Dense)            (None, 64)              

In [368]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

In [369]:
model.fit(X_train, Y_train, epochs=100, batch_size=40, validation_data=(X_Dev, Y_Dev), verbose=2, callbacks=[early_stopping])

Train on 1190 samples, validate on 298 samples
Epoch 1/100
1190/1190 - 1s - loss: 1.1490 - accuracy: 0.7496 - val_loss: 0.7069 - val_accuracy: 0.7919
Epoch 2/100
1190/1190 - 0s - loss: 0.6965 - accuracy: 0.7824 - val_loss: 0.5585 - val_accuracy: 0.7919
Epoch 3/100
1190/1190 - 0s - loss: 0.5862 - accuracy: 0.7815 - val_loss: 0.4753 - val_accuracy: 0.8121
Epoch 4/100
1190/1190 - 0s - loss: 0.5207 - accuracy: 0.8176 - val_loss: 0.4274 - val_accuracy: 0.8893
Epoch 5/100
1190/1190 - 0s - loss: 0.4803 - accuracy: 0.8471 - val_loss: 0.3974 - val_accuracy: 0.8792
Epoch 6/100
1190/1190 - 0s - loss: 0.4478 - accuracy: 0.8622 - val_loss: 0.3704 - val_accuracy: 0.8926
Epoch 7/100
1190/1190 - 0s - loss: 0.4221 - accuracy: 0.8731 - val_loss: 0.3376 - val_accuracy: 0.8993
Epoch 8/100
1190/1190 - 0s - loss: 0.4167 - accuracy: 0.8681 - val_loss: 0.3646 - val_accuracy: 0.8926
Epoch 9/100
1190/1190 - 0s - loss: 0.3967 - accuracy: 0.8756 - val_loss: 0.3297 - val_accuracy: 0.9161
Epoch 10/100
1190/1190 - 0

In [370]:
score = model.evaluate(X_test, Y_test)
print(score)

638/638 [==============================] - 0s 64us/sample - loss: 0.3271 - accuracy: 0.8903
[0.3270895482043861, 0.89028215]


In [229]:
model = keras.Sequential()
model.add(Dense(units=64, input_dim=X_train.shape[1], activation='tanh', kernel_initializer='zeros'))
model.add(Dense(units=3, activation='softmax'))

In [230]:
model.compile(loss='categorical_crossentropy', \
              optimizer='sgd', metrics=['accuracy'])

In [231]:
model.fit(X_train, Y_train, epochs=10, batch_size=1, verbose=0)

In [52]:
score = model.evaluate(X_test, Y_test)
print(score)

638/638 [==============================] - 0s 164us/sample - loss: 0.3257 - accuracy: 0.8668
[0.3256858137894574, 0.86677116]
